In [5]:
import pandas as pd
from tqdm import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import shutil
from functions import extractEanCodShopfarma, GeraDataframe, incluirPreco_PP

In [6]:
# Função que altera o preço shopfarma:
def alteraPrecoShopfarma(arquivo_xls, return_price=False, desconto_retorno=0.00):
    lst_error = []
    data = pd.read_excel(arquivo_xls, usecols='A,B,C,D,E,F,G,H')
    if return_price == False:
        data['preco_novo'] = data['preco_pp'].astype(int) - 1.01
    else:
        data['preco_novo'] = (data['preco_venda'] - (data['preco_venda'] * (desconto_retorno))).astype(int) - 0.01
    driver = webdriver.Chrome()
    driver.get("http://179.187.94.169:4647/sgfpod1/Cad_0020.pod")
    driver.implicitly_wait(15)
    sleep(1)
    driver.find_element(By.NAME, 'id_cod_usuario').send_keys(6)
    sleep(1)
    driver.find_element(By.NAME, 'nom_senha').send_keys('0788')
    sleep(1)
    driver.find_element(By.NAME, 'login').click()
    sleep(5)
    data = data
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        try:
            cod_shopfarma = int(row["codshopfarma"])
            preco_novo = row["preco_novo"]
            driver.find_element(By.NAME, 'cod_redbarraEntrada').clear()
            sleep(1)
            driver.find_element(By.NAME, 'cod_redbarraEntrada').send_keys(cod_shopfarma)
            driver.find_element(By.NAME, 'cod_redbarraEntrada').send_keys(Keys.RETURN)
            sleep(2)
            driver.find_element(By.LINK_TEXT, 'Preço').click()
            sleep(5)
            driver.find_element(By.NAME, 'vlr_venda').click()
            driver.find_element(By.NAME, 'vlr_venda').send_keys(Keys.BACKSPACE)
            sleep(1)
            driver.find_element(By.NAME, 'vlr_venda').send_keys(str(preco_novo).replace('.', ','))
            sleep(1)
            driver.find_element(By.NAME, 'vlr_venda').send_keys(Keys.RETURN)
            sleep(2)
            driver.find_element(By.NAME, 'salvar').send_keys(Keys.RETURN)
            sleep(2)
            driver.find_element(By.NAME, 'cod_redbarraEntrada').clear()
            sleep(3)
        except:
            print(f'Erro ao alterar o preço do produto: {cod_shopfarma}, o preço correto é {preco_novo}')
            lst_error.append(cod_shopfarma)
            sleep(3)
            driver.refresh()
    return data, lst_error

if return_price == True:
                driver.find_element(By.LINK_TEXT, 'Configurações').click()
                sleep(2)
                driver.find_element(By.NAME, 'nom_obsvenda').clear()
                sleep(2)

data, lst_error = alteraPrecoShopfarma('anticoncepcionais.xls')
data

lst_error

In [7]:
data_retorno, lst_error_retorno = alteraPrecoShopfarma('planilhas/anticoncepcionais_28022023.xls', return_price=True)
data_retorno

100%|██████████| 175/175 [52:38<00:00, 18.05s/it]


,codshopfarma,laboratorio,ean,produto,preco_venda,preco_compra,preco_pp,diferencapreco,preco_novo
0,112695,FQM GRUPO,7898040320683,"ADOLESS 0,060MG+0,015MG 28CP",37.99,28.33,34.53,3.46,36.99
1,108423,FQM GRUPO,7898040322137,"ADOLESS 0,060MG+0,015MG 84CP",94.99,70.87,86.37,8.62,93.99
2,13094,SANOFI,7896422532228,"AIXA 2MG+0,03MG 63CP REV",135.99,94.38,129.13,6.86,134.99
3,14420,LEGRAND,7894916142809,ALGESTONA AC+ENAN EST INJ 1ML,10.99,9.11,11.12,-0.13,9.99
4,129781,ACHE,7896658035975,ALLESTRA 15 60MCG+15MCG 28CP R,20.99,16.15,19.81,1.18,19.99
...,...,...,...,...,...,...,...,...,...
170,129802,BAYER SA,7891106914109,"YASMIN 3MG+0,03MG 63CP REV",255.99,176.58,243.44,12.55,254.99
171,129801,BAYER SA,7891106914130,"YASMIN 3MG+0,03MG 63CP REV",255.99,197.37,243.44,12.55,254.99
172,129803,BAYER SA,7891106914154,"YAZ 3MG+0,02MG 24+4CP REV",84.99,65.81,81.40,3.59,83.99
173,10741,BAYER SA,7891106914147,"YAZ 3MG+0,02MG 24CP REV",84.99,58.87,85.91,-0.92,83.99


In [6]:
data_retorno.sort_values(by=['produto'], ascending=True)

,codshopfarma,laboratorio,ean,produto,preco_venda,preco_compra,preco_pp,diferencapreco,preco_novo
0,112695,FQM GRUPO,7898040320683,"ADOLESS 0,060MG+0,015MG 28CP",37.99,28.33,34.53,3.46,32.99
1,108423,FQM GRUPO,7898040322137,"ADOLESS 0,060MG+0,015MG 84CP",94.99,70.87,86.37,8.62,84.99
2,13094,SANOFI,7896422532228,"AIXA 2MG+0,03MG 63CP REV",135.99,94.38,129.13,6.86,127.99
3,14420,LEGRAND,7894916142809,ALGESTONA AC+ENAN EST INJ 1ML,10.99,9.11,11.12,-0.13,9.99
4,129781,ACHE,7896658035975,ALLESTRA 15 60MCG+15MCG 28CP R,20.99,16.15,19.81,1.18,17.99
...,...,...,...,...,...,...,...,...,...
170,129802,BAYER SA,7891106914109,"YASMIN 3MG+0,03MG 63CP REV",255.99,176.58,243.44,12.55,241.99
171,129801,BAYER SA,7891106914130,"YASMIN 3MG+0,03MG 63CP REV",255.99,197.37,243.44,12.55,241.99
172,129803,BAYER SA,7891106914154,"YAZ 3MG+0,02MG 24+4CP REV",84.99,65.81,81.40,3.59,79.99
173,10741,BAYER SA,7891106914147,"YAZ 3MG+0,02MG 24CP REV",84.99,58.87,85.91,-0.92,83.99


In [47]:
def ajustaValor(dataframe, desconto=0.00):
    lst_error = []
    driver = webdriver.Chrome()
    driver.get("http://177.204.38.180:4647/sgfpod1/Cad_0020.pod")
    driver.implicitly_wait(15)
    sleep(1)
    driver.find_element(By.NAME, 'id_cod_usuario').send_keys(6)
    sleep(1)
    driver.find_element(By.NAME, 'nom_senha').send_keys(1711)
    sleep(1)
    driver.find_element(By.NAME, 'login').click()
    sleep(5)
    data = dataframe
    for index, row in tqdm(data.iterrows()):
        try:
            cod_shopfarma = int(row["codshopfarma"])
            driver.find_element(By.NAME, 'cod_redbarraEntrada').clear()
            sleep(1)
            driver.find_element(By.NAME, 'cod_redbarraEntrada').send_keys(cod_shopfarma)
            driver.find_element(By.NAME, 'cod_redbarraEntrada').send_keys(Keys.RETURN)
            sleep(2)
            driver.find_element(By.LINK_TEXT, 'Preço').click()
            sleep(5)
            preco_pmc = driver.find_element(By.NAME, 'vlr_pmc').get_attribute('value')
            preco_pmc = float(preco_pmc.replace(',', '.'))
            preco_novo = int(preco_pmc - (preco_pmc * desconto)) - 0.01
            sleep(2)
            driver.find_element(By.NAME, 'vlr_venda').click()
            driver.find_element(By.NAME, 'vlr_venda').send_keys(Keys.BACKSPACE)
            sleep(1)
            driver.find_element(By.NAME, 'vlr_venda').send_keys(str(preco_novo).replace('.', ','))
            sleep(1)
            driver.find_element(By.NAME, 'vlr_venda').send_keys(Keys.RETURN)
            sleep(2)
            driver.find_element(By.NAME, 'salvar').send_keys(Keys.RETURN)
            sleep(2)
            driver.find_element(By.NAME, 'cod_redbarraEntrada').clear()
            sleep(3)   
            driver.refresh() 
            desconto = round((1-preco_novo/preco_pmc)*100, 2)
            print(f'Preço do produto: {cod_shopfarma}, alterado para {preco_novo}, o preço PMC é {preco_pmc}. Desconto de {desconto}%')
        except:
            try:
                print(f'Erro ao alterar o preço do produto: {cod_shopfarma}, o preço PMC é {preco_pmc}')
                lst_erro = cod_shopfarma, preco_pmc
                lst_error.append(lst_erro)
            except:
                print(f'Erro ao encontrar o PMC do produto: {preco_pmc}')
                lst_erro = cod_shopfarma, 0
                lst_error.append(lst_erro)
            sleep(3)
            driver.refresh()

In [49]:
ajustaValor(data_retorno, desconto=0.15)

1it [00:20, 20.79s/it]

Preço do produto: 5793, alterado para 43.99, o preço PMC é 52.91. Desconto de 16.85881685881685%


2it [00:41, 20.82s/it]

Preço do produto: 107556, alterado para 17.99, o preço PMC é 21.93. Desconto de 17.96625626994984%


3it [01:02, 20.84s/it]

...

Preço do produto: 13146, alterado para 7.99, o preço PMC é 10.22. Desconto de 21.819960861056753%


175it [57:12, 20.02s/it]

Preço do produto: 102662, alterado para 7.99, o preço PMC é 10.51. Desconto de 23.977164605137958%


176it [57:32, 19.62s/it]

Preço do produto: 13107, alterado para 28.99, o preço PMC é 34.57. Desconto de 16.141162857969338%
